# Pre - Processament de Dades Meteorologiques dels episodis i temporals adversos del Delta del Ebre

| Data d'inici  | Data final   | Esdeveniment                                       |
|---------------|--------------|-----------------------------------------------------|
| 21/11/2016    | 03/12/2016   | Altes Precipitacions, novembre 2016                 |
| 14/01/2017    | 28/01/2017   | Temporal Bruno (alta precipitació y vent), Gener 2017|
| 13/10/2017    | 24/10/2017   | Alta Precipitación, Octubre 2017                    |
| 28/12/2017    | 09/01/2018   | Ventada, Gener 2018                                 |
| 09/10/2018    | 20/10/2018   | Ex Huracà Leslie (Ventada), Octubre 2018            |
| 13/10/2018    | 25/10/2018   | Aiguats a l'Ebre (Altes precipitacions), Octubre 2018|
| 09/11/2018    | 21/11/2018   | Alta precipitació, Novembre 2018                    |
| 27/01/2019    | 08/02/2019   | Ventada, Febrer 2019                                |
| 17/10/2019    | 28/10/2019   | Altes precipitacions, octubre 2019                  |
| 28/11/2019    | 10/12/2019   | Altes precipitacions i vent, desembre 2019          |
| 14/01/2020    | 26/01/2020   | Temporal Gloria (Altes precipitacions i vent, gener 2020)|
| 27/08/2020    | 08/09/2020   | Aiguats Alcanar (Altes precipitacions), setembre 2020|
| 06/12/2020    | 17/12/2020   | Tornado Horta de Sant Joan, (vent al delta), desembre 2020|
| 24/03/2020    | 07/04/2020   | Altes precipitacions, març 2020                    |
| 04/01/2021    | 17/01/2021   | Temporal Filomena (Altes precipitacions), gener 2021|
| 17/11/2021    | 30/11/2021   | Altes precipitacions i vent (Tornado), novembre 2021|



Estacions estudiades:

* U9: L'Aldea
* DL: Sant Jaume d'Enveja
* UU: Amposta
* UW: La Ràpita

Altres estacions mes allunyades del Delta del Ebre:

* UA: L'Ametlla de Mar
* U7: Aldover
* X5: Roquetes
* C9: Mas de Barberans
* US: Alcanar
* UT: les Cases d'Alcanar
* UX: Ulldecona - els Valentins

## LLibreríes

In [23]:
# Importar llibreries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys
import time
import datetime 
import cartopy.crs as ccrs

## Codis, variables, dates i directoris

In [24]:
# Establim el directori de treball (detectar el path principal)

path_principal = os.getcwd()
carpeta_dades = 'Dades SCM Temporals Adversos Delta Ebre'
path_dades = os.path.join(path_principal, carpeta_dades)

carpeta_metadades = 'Metadades i codis'
metadates_file = 'SCM_metadades_diaries_mensuals.json'
metadates_path = os.path.join(path_principal, carpeta_metadades, metadates_file)

In [25]:
# LLista estacions amb codi, estacio, latitud i longitud

estacions =  [
        ['U9', 'L\'Aldea', 40.76947, 0.61635],
        ['DL', 'Sant Jaume d\'Enveja', 40.70719, 0.83449],
        ['UU', 'Amposta', 40.70776, 0.63210],
        ['UW', 'La Ràpita', 40.62725, 0.65922],
        ['UA','L\'Ametlla de Mar', 40.9091, 0.76943],
        ['U7','Aldover',40.85936,0.50525],
        ['X5','Roquetes',40.79705,0.31822],
        ['C9','Mas de Barberans',40.71825,0.39988],
        ['US','Alcanar',40.55786,0.52329],
        ['UT','les Cases d\'Alcanar',40.55219,0.53252],
        ['UX','Ulldecona - els Valentins',40.62652,0.37112]
        ]

# Diccionari amb els codis de les variables

codis_variables = {
                1300 : 'Precipitació acumulada diària (mm)', 
                1503 : 'Velocitat mitjana diària del vent 10 m (m/s)', 
                1505 : 'Velocitat mitjana diària del vent 2 m (m/s)',
                1509 : 'Direcció mitjana diària del vent 10 m (graus)', 
                1511 : 'Direcció mitjana diària del vent 2 m (s/m)', 
                1512 : 'Ratxa màxima diària del vent 10 m (m/s)', 
                1514 : 'Ratxa màxima diària del vent 2 m (m/s)', 
                1515 : 'Direcció de la ratxa màxima diària del vent 10 m (graus)', 
                1517 : 'Direcció de la ratxa màxima diària del vent 2 m (graus)'
                }

# Creem la llista de dates a partir de la data inicial i final

events = [
    ["26/11/2016", "28/11/2016", "Altes Precipitacions, novembre 2016"],
    ["19/01/2017", "23/01/2017", "Temporal Bruno (alta precipitació y vent), Gener 2017"],
    ["18/10/2017", "19/10/2017", "Alta Precipitación, Octubre 2017"],
    ["02/01/2018", "04/01/2018", "Ventada, Gener 2018"],
    ["14/10/2018", "15/10/2018", "Ex Huracà Leslie (Ventada), Octubre 2018"],
    ["18/10/2018", "20/10/2018", "Aiguats a l'Ebre (Altes precipitacions), Octubre 2018"],
    ["14/11/2018", "16/11/2018", "Alta precipitació, Novembre 2018"],
    ["01/02/2019", "03/02/2019", "Ventada, Febrer 2019"],
    ["22/10/2019", "23/10/2019", "Altes precipitacions, octubre 2019"],
    ["03/12/2019", "05/12/2019", "Altes precipitacions i vent, desembre 2019"],
    ["19/01/2020", "21/01/2020", "Temporal Gloria (Altes precipitacions i vent, gener 2020)"],
    ["01/09/2020", "03/09/2020", "Aiguats Alcanar (Altes precipitacions), setembre 2020"],
    ["11/12/2020", "12/12/2020", "Tornado Horta de Sant Joan, (vent al delta), desembre 2020"],
    ["29/03/2020", "02/04/2020", "Altes precipitacions, març 2020"],
    ["09/01/2021", "12/01/2021", "Temporal Filomena (Altes precipitacions), gener 2021"],
    ["22/11/2021", "25/11/2021", "Altes precipitacions i vent (Tornado), novembre 2021"]
]


# Llista amb els events meteorologics importants
ext_events = [
        ['21/11/2016', '03/12/2016', 'Altes Precipitacions, novembre 2016'],
        ['14/01/2017', '28/01/2017', 'Temporal Bruno (alta precipitació y vent), Gener 2017'],
        ['13/10/2017', '24/10/2017', 'Alta Precipitación, Octubre 2017'],
        ['28/12/2017', '09/01/2018', 'Ventada, Gener 2018'],
        ['09/10/2018', '20/10/2018', 'Ex Huracà Leslie (Ventada), Octubre 2018'],
        ['13/10/2018', '25/10/2018', "Aiguats a l'Ebre (Altes precipitacions), Octubre 2018"],
        ['09/11/2018', '21/11/2018', 'Alta precipitació, Novembre 2018'], 
        ['27/01/2019', '08/02/2019', 'Ventada, Febrer 2019'], 
        ['17/10/2019', '28/10/2019', 'Altes precipitacions, octubre 2019'], 
        ['28/11/2019', '10/12/2019', 'Altes precipitacions i vent, desembre 2019'], 
        ['14/01/2020', '26/01/2020', 'Temporal Gloria (Altes precipitacions i vent, gener 2020)'], 
        ['27/08/2020', '08/09/2020', 'Aiguats Alcanar (Altes precipitacions), setembre 2020'], 
        ['06/12/2020', '17/12/2020', 'Tornado Horta de Sant Joan, (vent al delta), desembre 2020'], 
        ['24/03/2020', '07/04/2020', 'Altes precipitacions, març 2020'], 
        ['04/01/2021', '17/01/2021', 'Temporal Filomena (Altes precipitacions), gener 2021'], 
        ['17/11/2021', '30/11/2021', 'Altes precipitacions i vent (Tornado), novembre 2021']
]


In [26]:
# Passar les variables llistes anteriors a dataframes

events_df = pd.DataFrame(events, columns = ['data inici', 'data fi', 'event'])
ext_events_df = pd.DataFrame(ext_events, columns = ['data inici ext', 'data fi ext', 'event'])
codis_variables_df = pd.DataFrame(codis_variables.items(), columns = ['codi', 'variable'])
# estacions_df = pd.DataFrame(estacions.items(), columns = ['codi', 'estacio'])
estacions_df = pd.DataFrame(estacions, columns = ['codi', 'estacio', 'latitud', 'longitud'])


# Passar les dates a format datetime

events_df['data inici'] = pd.to_datetime(events_df['data inici'], format = '%d/%m/%Y')
events_df['data fi'] = pd.to_datetime(events_df['data fi'], format = '%d/%m/%Y')
ext_events_df['data inici ext'] = pd.to_datetime(ext_events_df['data inici ext'], format = '%d/%m/%Y')
ext_events_df['data fi ext'] = pd.to_datetime(ext_events_df['data fi ext'], format = '%d/%m/%Y')


# Ampliar tota la informació de dates en el dataframe events extens

ext_events_df = pd.concat([events_df[['data inici', 'data fi']], ext_events_df], axis=1)

In [27]:

# Exportem els dataframes a csv al metadates path per poder-los utilitzar en altres scripts, juntament amb el diccionari de variables. Tot a csv perque es mes facil de llegir per altres scripts 

events_df.to_csv(os.path.join(carpeta_metadades, 'events.csv'), index = False)
ext_events_df.to_csv(os.path.join(carpeta_metadades, 'ext_events.csv'), index = False)
estacions_df.to_csv(os.path.join(carpeta_metadades, 'estacions.csv'), index = False)
codis_variables_df.to_csv(os.path.join(carpeta_metadades, 'codis_variables.csv'), index = False)


## Carregar Metadades i filtrar les necessaries

In [28]:
# Carreguem les metadades des del fitxer json per entendre quines variables tenim, les unitats, etc.

with open(metadates_path , 'r') as f:
    metadades = json.load(f)
# Organitzar les metadata en un dataframe

metadades_df = pd.DataFrame(metadades)
metadades_df

# Eliminem les metadates referents a variables que no apareixen a la llista de codis de variables
metadades_df = metadades_df[metadades_df['codi'].isin(codis_variables.keys())].reset_index(drop = True)

# guardem el dataframe de metadades seleccionades a un csv per poder-lo utilitzar en altres scripts

metadades_df.to_csv(os.path.join(carpeta_metadades, 'metadades_selec.csv'), index = False)

metadades_df

,codi,nom,unitat,acronim,tipus,decimals
0,1300,Precipitació acumulada diària,mm,PPT,AD,1
1,1503,Velocitat mitjana diària del vent 10 m (esc.),m/s,VVM10,AD,1
2,1505,Velocitat mitjana diària del vent 2 m (esc.),m/s,VVM2,AD,1
3,1509,Direcció mitjana diària del vent 10 m (m. 1),°,DVM10,AD,0
4,1511,Direcció mitjana diària del vent 2 m (m. 1),°,DVM2,AD,0
5,1512,Ratxa màxima diària del vent 10 m + hora,m/s,VVX10,AD,1
6,1514,Ratxa màxima diària del vent 2 m + hora,m/s,VVX2,AD,1
7,1515,Direcció de la ratxa màx. diària de vent 10 m,°,DVVX10,AD,0
8,1517,Direcció de la ratxa màx. diària de vent 2 m,°,DVVX2,AD,0


In [29]:
# Mostrem el dataframe de les estacions
estacions_df

,codi,estacio,latitud,longitud
0,U9,L'Aldea,40.76947,0.61635
1,DL,Sant Jaume d'Enveja,40.70719,0.83449
2,UU,Amposta,40.70776,0.63210
3,UW,La Ràpita,40.62725,0.65922
4,UA,L'Ametlla de Mar,40.90910,0.76943
5,U7,Aldover,40.85936,0.50525
6,X5,Roquetes,40.79705,0.31822
7,C9,Mas de Barberans,40.71825,0.39988
8,US,Alcanar,40.55786,0.52329
9,UT,les Cases d'Alcanar,40.55219,0.53252


In [30]:
# Mostrem el dataframe de les dates dels esdeveniments
ext_events_df

,data inici,data fi,data inici ext,data fi ext,event
0,2016-11-26,2016-11-28,2016-11-21,2016-12-03,"Altes Precipitacions, novembre 2016"
1,2017-01-19,2017-01-23,2017-01-14,2017-01-28,"Temporal Bruno (alta precipitació y vent), Gen..."
2,2017-10-18,2017-10-19,2017-10-13,2017-10-24,"Alta Precipitación, Octubre 2017"
3,2018-01-02,2018-01-04,2017-12-28,2018-01-09,"Ventada, Gener 2018"
4,2018-10-14,2018-10-15,2018-10-09,2018-10-20,"Ex Huracà Leslie (Ventada), Octubre 2018"
5,2018-10-18,2018-10-20,2018-10-13,2018-10-25,"Aiguats a l'Ebre (Altes precipitacions), Octub..."
6,2018-11-14,2018-11-16,2018-11-09,2018-11-21,"Alta precipitació, Novembre 2018"
7,2019-02-01,2019-02-03,2019-01-27,2019-02-08,"Ventada, Febrer 2019"
8,2019-10-22,2019-10-23,2019-10-17,2019-10-28,"Altes precipitacions, octubre 2019"
9,2019-12-03,2019-12-05,2019-11-28,2019-12-10,"Altes precipitacions i vent, desembre 2019"


## Carregar les primeres dades i tractament previ dels dataframes (cas preliminar per escalar)

In [31]:
# Fem primer el bolcat de dades de l'episodi del temporal Gloria, després ja ho escalarem a tots els episodis

# Nome event temporal Gloria
nom_event = ext_events[10][2]

# Accedim a la carpeta de dades de l'episodi
path_event = os.path.join(path_dades, nom_event)

# Llistem els fitxers de la carpeta
data_files = os.listdir(path_event)

# Creem un dataframe buit on anirem afegint les dades de cada fitxer
event_df = pd.DataFrame()

# Iterem sobre els fitxers json de la carpeta

# for file in data_files:
# Obtenim el path del fitxer
file_path = os.path.join(path_event, data_files[0])
# Obrim el fitxer json
with open(file_path, 'r') as f:
    # Carreguem el json a un diccionari
    data = json.load(f)
    # Convertim el diccionari a un dataframe
    data_df = pd.DataFrame(data)
       

In [32]:
data_df.head(5)

,codiEstacio,codiVariable,valors
0,C9,1300,"{'data': '2020-01-01Z', 'valor': 0, 'percentat..."
1,C9,1300,"{'data': '2020-01-02Z', 'valor': 0, 'percentat..."
2,C9,1300,"{'data': '2020-01-03Z', 'valor': 0, 'percentat..."
3,C9,1300,"{'data': '2020-01-04Z', 'valor': 0, 'percentat..."
4,C9,1300,"{'data': '2020-01-05Z', 'valor': 0, 'percentat..."


In [33]:
# Reestructurem la columna valors per tenir una columna per cada variable
print(data_df['valors'][0])
print(type(data_df['valors'][0]))

{'data': '2020-01-01Z', 'valor': 0, 'percentatge': 100}
<class 'dict'>


In [34]:
# Desgranar el string de la columna valors per obtenir una columna per cada variable, afegir-ho al dataframe i eliminar la columna valors
data_df = pd.concat([data_df.drop(['valors'], axis=1), data_df['valors'].apply(pd.Series)], axis=1)
data_df.head(5)

,codiEstacio,codiVariable,data,valor,percentatge
0,C9,1300,2020-01-01Z,0.0,100
1,C9,1300,2020-01-02Z,0.0,100
2,C9,1300,2020-01-03Z,0.0,100
3,C9,1300,2020-01-04Z,0.0,100
4,C9,1300,2020-01-05Z,0.0,100


In [35]:
# Convertir la columna data a format datetime
# Primer eliminem la Z al final de la cadena
data_df['data'] = data_df['data'].str.replace('Z', '')

# Convertim la columna data a format datetime
data_df['data'] = pd.to_datetime(data_df['data'], format= '%Y-%m-%d')

data_df.head(15)

,codiEstacio,codiVariable,data,valor,percentatge
0,C9,1300,2020-01-01,0.0,100
1,C9,1300,2020-01-02,0.0,100
2,C9,1300,2020-01-03,0.0,100
3,C9,1300,2020-01-04,0.0,100
4,C9,1300,2020-01-05,0.0,100
5,C9,1300,2020-01-06,0.0,100
6,C9,1300,2020-01-07,0.0,100
7,C9,1300,2020-01-08,0.1,100
8,C9,1300,2020-01-09,0.0,100
9,C9,1300,2020-01-10,0.0,100


## Carregar les dades de les altres variables i estacions, ordenar-les i concatenar-les

In [36]:
# Repeteix el mateix per tots els fitxers de la carpeta i afegir-ho al dataframe event_df
# Iterem sobre els fitxers json de la carpeta

nom_event = ext_events[10][2]
path_event = os.path.join(path_dades, nom_event)

# Llistem els fitxers de la carpeta
data_files = os.listdir(path_event)

event_df = pd.DataFrame()

for file in data_files:
    file_path = os.path.join(path_event, file)
    # Obrim el fitxer json
    with open (file_path,'r') as f:
        # Carreguem el json a un diccionari
        data = json.load(f)
        # Convertim el diccionari a un dataframe i afegim-ho al dataframe event_df
        event_df = pd.concat([event_df, pd.DataFrame(data)], ignore_index=True)


        
# Restructurem la columna valors per tenir una columna per cada variable
event_df = pd.concat([event_df.drop(['valors'], axis=1), event_df['valors'].apply(pd.Series)], axis=1)
# Convertir la columna data a format datetime
# Primer eliminem la Z al final de la cadena

event_df['data'] = event_df['data'].str.replace('Z', '')

# Convertim la columna data a format datetime
event_df['data'] = pd.to_datetime(event_df['data'], format= '%Y-%m-%d')
# Convertim la columna dataExtrem en format datetime, en aquest cas de hores, ja que fa referencia a l'hora de l'extrem

# DataExtrem te aquest format "2020-01-01T07:58Z"
event_df['dataExtrem'] = event_df['dataExtrem'].str.replace('Z', '')
event_df['dataExtrem'] = pd.to_datetime(event_df['dataExtrem'], format= '%Y-%m-%dT%H:%M')

# Eliminem dela columna dataExtrem la part dels dies, ja que no ens interessa
event_df['dataExtrem'] = event_df['dataExtrem'].dt.time
#Eliminem la columna percentatge ja que no ens aporta res
event_df = event_df.drop(['percentatge'], axis=1) 

# Renombrar les columnes dataExtrem segons si es la hora de la racha de 10m o 2m

# Crear una columna nova anomenada HE10M i HE2M

# Si el codi variable es 1512 afegir el valor de dataExtrem a la columna HE10M
# Si el codi variable es 1514 afegir el valor de dataExtrem a la columna HE2M
# Eliminar la columna dataExtrem
# Crear una columna nova anomenada HE10M i HE2M

event_df['HE10M'] = np.nan
event_df['HE2M'] = np.nan

# Si el codi variable es 1512 afegir el valor de dataExtrem a la columna HE10M
# Si el codi variable es 1514 afegir el valor de dataExtrem a la columna HE2M

event_df.loc[event_df['codiVariable'] == 1512, 'HE10M'] = event_df['dataExtrem']
event_df.loc[event_df['codiVariable'] == 1514, 'HE2M'] = event_df['dataExtrem']

# Eliminar la columna dataExtrem
event_df = event_df.drop(['dataExtrem'], axis=1)

# Renombrem els valors de la columna codiVariable per tenir l'acronim de la variable segons els acronims del dataframe metadades_df
event_df['codiVariable'] = event_df['codiVariable'].replace(metadades_df.set_index('codi')['acronim'].to_dict())


# Cambiar l'ordre de les columnes

event_df = event_df[['data','codiEstacio','codiVariable','valor','HE10M','HE2M']]


# Les variables VVX2 i VVX10 tenen un valor extra en la columna HE2M i HE10M respectivament, ja que en la importació de dades incloia la data i l'hora de l'extrem.
# El nom de les columnes s'ha posat manualment (no venia en les metadades) simulant com si fos una variable mes, en comptes de 2 en 1.
# Hem d'aconseguir elimnar les columnes HE2M i HE10M i en els casos que el valor no sigui NaN, afegir-lo com a una nova fila amb el valor a la columna variable i el codiVariable el nom de la columna HE2M o HE10M, conservant el el valor de la columa data i codiEstacio, despres pivotarem tot el data frame per tenir una columna per cada variable i una fila per cada dia i estacio.

# Creem un nou dataframe on nomes agafem les columnes data, codiEstacio, codiVariable, valor, HE10M i HE2M
event_df_vent = event_df[['data','codiEstacio','codiVariable','valor','HE10M','HE2M']]
# Elimiem les files on HE10M i HE2M son NaN (les dues columnes a la vegada)
event_df_vent = event_df_vent.dropna(subset=['HE10M', 'HE2M'], how='all')


# Buidem tots els valors de la columna 'valor'
event_df_vent['valor'] = np.nan

# Canviem el valor de la columa codiVariable per el valor de la columna HE10M o HE2M segons el cas
# HE10M si es VVX10 i HE2M si es VVX2
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX10', 'valor'] = event_df_vent['HE10M']
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX2', 'valor'] = event_df_vent['HE2M']

# Substituir el valor de la columna codiVariable per HE10M o HE2M segons el cas
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX10', 'codiVariable'] = 'HE10M'
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX2', 'codiVariable'] = 'HE2M'

# Eliminem les columnes HE10M i HE2M
event_df_vent = event_df_vent.drop(['HE10M', 'HE2M'], axis=1)

# Afegim el dataframe event_df_vent al dataframe event_df
event_df = pd.concat([event_df, event_df_vent], ignore_index=True)
del event_df_vent

# Eliminar les columnes HE10M i HE2M
event_df = event_df.drop(['HE10M', 'HE2M'], axis=1)

# Ordenar el dataframe per data i codiEstacio
event_df = event_df.sort_values(['data', 'codiEstacio']).reset_index(drop = True)

event_df.head(10)


,data,codiEstacio,codiVariable,valor
0,2020-01-01,C9,PPT,0.0
1,2020-01-01,C9,VVM10,1.1
2,2020-01-01,C9,DVM10,35.0
3,2020-01-01,C9,VVX10,4.2
4,2020-01-01,C9,DVVX10,27.0
5,2020-01-01,C9,HE10M,12:05:00
6,2020-01-01,DL,PPT,0.0
7,2020-01-01,DL,VVM10,3.4
8,2020-01-01,DL,DVM10,328.0
9,2020-01-01,DL,VVX10,10.2


In [37]:
# Pivotem la Taula per tenir una columna per cada variable i una fila per cada dia i estacio

event_df = event_df.pivot(index=['data', 'codiEstacio'], columns='codiVariable', values='valor')
event_df.head(10)

codiVariable            DVM10   DVM2 DVVX10  DVVX2     HE10M      HE2M  PPT  \
data       codiEstacio                                                        
2020-01-01 C9            35.0    NaN   27.0    NaN  12:05:00       NaN  0.0   
           DL           328.0    NaN  328.0    NaN  07:58:00       NaN  0.0   
           U7             NaN  336.0    NaN    5.0       NaN  09:38:00  0.0   
           U9             NaN  288.0    NaN  325.0       NaN  08:12:00  0.0   
           UA             NaN  298.0    NaN  296.0       NaN  10:45:00  0.0   
           US             NaN  300.0    NaN  110.0       NaN  11:45:00  0.0   
           UU           289.0    NaN  300.0    NaN  10:26:00       NaN  0.0   
           UW             NaN  330.0    NaN  332.0       NaN  10:46:00  0.0   
           UX             NaN   40.0    NaN   42.0       NaN  12:00:00  0.0   
           X5           265.0    NaN  312.0    NaN  05:23:00       NaN  0.0   

codiVariable           VVM10 VVM2 VVX10 VVX2  
data       codiEstacio                        
2020-01-01 C9            1.1  NaN   4.2  NaN  
           DL            3.4  NaN  10.2  NaN  
           U7            NaN  2.8   NaN  8.9  
           U9            NaN  0.6   NaN  4.1  
           UA            NaN  1.8   NaN  7.7  
           US            NaN  0.6   NaN  2.8  
           UU            3.8  NaN   8.7  NaN  
           UW            NaN  3.5   NaN  8.3  
           UX            NaN  0.7   NaN  3.6  
           X5            3.2  NaN  12.7  NaN

## Carregar i Netejar les dades de totes les estacions, dates i episodis, totes les carpetes (cas escalat) i guardat a un unic dataframe amb tot per treballar-hi

In [38]:
# Creem els path principals

path_principal = os.getcwd()
carpeta_dades = 'Dades SCM Temporals Adversos Delta Ebre'
path_dades = os.path.join(path_principal, carpeta_dades)

# Bolquem les dades de tots els arxius i totes les carpetes a un dataframe

event_data_all_df = pd.DataFrame()

# Llista d'events (carpetes) que es igual variable llista
events_folders = os.listdir(path_dades)

for event in events_folders: 
    data_files = os.listdir(os.path.join(path_dades, event))
    
    for file in data_files:
        file_path = os.path.join(path_dades, event, file)

        # Obrim el fitxer json
        with open (file_path,'r') as f:
            # Carreguem el json a un diccionari
            data = json.load(f)
            # Convertim el diccionari a un dataframe i afegim-ho al dataframe event_data_all_df
            event_data_all_df = pd.concat([event_data_all_df, pd.DataFrame(data)], ignore_index=True)

            
# Restructurem la columna valors per tenir una columna per cada variable
event_data_all_df = pd.concat([event_data_all_df.drop(['valors'], axis=1), event_data_all_df['valors'].apply(pd.Series)], axis=1)
# Convertir la columna data a format datetime
# Primer eliminem la Z al final de la cadena

event_data_all_df['data'] = event_data_all_df['data'].str.replace('Z', '')

# Convertim la columna data a format datetime
event_data_all_df['data'] = pd.to_datetime(event_data_all_df['data'], format= '%Y-%m-%d')

# Convertim la columna dataExtrem en format datetime, en aquest cas de hores, ja que fa referencia a l'hora de l'extrem
# DataExtrem te aquest format "2020-01-01T07:58Z"
event_data_all_df['dataExtrem'] = event_data_all_df['dataExtrem'].str.replace('Z', '')
event_data_all_df['dataExtrem'] = pd.to_datetime(event_data_all_df['dataExtrem'], format= '%Y-%m-%dT%H:%M')

# Eliminem dela columna dataExtrem la part dels dies, ja que no ens interessa
event_data_all_df['dataExtrem'] = event_data_all_df['dataExtrem'].dt.time
#Eliminem la columna percentatge ja que no ens aporta res
event_data_all_df = event_data_all_df.drop(['percentatge'], axis=1) 

# Renombrar les columnes dataExtrem segons si es la hora de la racha de 10m o 2m

# Crear una columna nova anomenada HE10M i HE2M

# Si el codi variable es 1512 afegir el valor de dataExtrem a la columna HE10M
# Si el codi variable es 1514 afegir el valor de dataExtrem a la columna HE2M
# Eliminar la columna dataExtrem
# Crear una columna nova anomenada HE10M i HE2M

event_data_all_df['HE10M'] = np.nan
event_data_all_df['HE2M'] = np.nan

# Si el codi variable es 1512 afegir el valor de dataExtrem a la columna HE10M
# Si el codi variable es 1514 afegir el valor de dataExtrem a la columna HE2M

event_data_all_df.loc[event_data_all_df['codiVariable'] == 1512, 'HE10M'] = event_data_all_df['dataExtrem']
event_data_all_df.loc[event_data_all_df['codiVariable'] == 1514, 'HE2M'] = event_data_all_df['dataExtrem']

# Eliminar la columna dataExtrem
event_data_all_df = event_data_all_df.drop(['dataExtrem'], axis=1)

# Renombrem els valors de la columna codiVariable per tenir l'acronim de la variable segons els acronims del dataframe metadades_df
event_data_all_df['codiVariable'] = event_data_all_df['codiVariable'].replace(metadades_df.set_index('codi')['acronim'].to_dict())

# Cambiar l'ordre de les columnes

event_data_all_df = event_data_all_df[['data','codiEstacio','codiVariable','valor','HE10M','HE2M']]


# Les variables VVX2 i VVX10 tenen un valor extra en la columna HE2M i HE10M respectivament, ja que en la importació de dades incloia la data i l'hora de l'extrem.
# El nom de les columnes s'ha posat manualment (no venia en les metadades) simulant com si fos una variable mes, en comptes de 2 en 1.
# Hem d'aconseguir elimnar les columnes HE2M i HE10M i en els casos que el valor no sigui NaN, afegir-lo com a una nova fila amb el valor a la columna variable i el codiVariable el nom de la columna HE2M o HE10M, conservant el el valor de la columa data i codiEstacio, despres pivotarem tot el data frame per tenir una columna per cada variable i una fila per cada dia i estacio.

# Creem un nou dataframe on nomes agafem les columnes data, codiEstacio, codiVariable, valor, HE10M i HE2M
event_df_vent = event_data_all_df[['data','codiEstacio','codiVariable','valor','HE10M','HE2M']]
# Elimiem les files on HE10M i HE2M son NaN (les dues columnes a la vegada)
event_df_vent = event_df_vent.dropna(subset=['HE10M', 'HE2M'], how='all')


# Buidem tots els valors de la columna 'valor'
event_df_vent['valor'] = np.nan

# Canviem el valor de la columa codiVariable per el valor de la columna HE10M o HE2M segons el cas
# HE10M si es VVX10 i HE2M si es VVX2
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX10', 'valor'] = event_df_vent['HE10M']
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX2', 'valor'] = event_df_vent['HE2M']

# Substituir el valor de la columna codiVariable per HE10M o HE2M segons el cas
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX10', 'codiVariable'] = 'HE10M'
event_df_vent.loc[event_df_vent['codiVariable'] == 'VVX2', 'codiVariable'] = 'HE2M'

# Eliminem les columnes HE10M i HE2M
event_df_vent = event_df_vent.drop(['HE10M', 'HE2M'], axis=1)

# Afegim el dataframe event_df_vent al dataframe event_data_all_df
event_data_all_df = pd.concat([event_data_all_df, event_df_vent], ignore_index=True)
del event_df_vent

# Eliminar les columnes HE10M i HE2M
event_data_all_df = event_data_all_df.drop(['HE10M', 'HE2M'], axis=1)

# Eliminar els elements duplicats, ja que hi ha alguns episodis que passen al mateix mes i per tant les dades estan descarregades i importades per duplicat

# Eliminar entradas duplicadas
event_data_all_df = event_data_all_df.drop_duplicates(['data', 'codiEstacio', 'codiVariable'], keep='first').reset_index(drop = True)


# Ordenar el dataframe per data i codiEstacio
event_data_all_df = event_data_all_df.sort_values(['data', 'codiEstacio']).reset_index(drop = True)

# Pivotem la Taula per tenir una columna per cada variable i una fila per cada dia i estacio
event_data_all_df = event_data_all_df.pivot(index=['data', 'codiEstacio'], columns='codiVariable', values='valor')

# Mostrem el dataframe final
event_data_all_df

codiVariable            DVM10   DVM2 DVVX10  DVVX2     HE10M      HE2M  PPT  \
data       codiEstacio                                                        
2016-11-01 C9           223.0    NaN  181.0    NaN  17:27:00       NaN  0.0   
           DL           273.0    NaN  176.0    NaN  14:43:00       NaN  0.0   
           U7             NaN    3.0    NaN    9.0       NaN  08:23:00  0.0   
           U9             NaN  268.0    NaN  143.0       NaN  14:06:00  0.0   
           UA             NaN  288.0    NaN  354.0       NaN  08:57:00  0.0   
...                       ...    ...    ...    ...       ...       ...  ...   
2021-11-30 US             NaN  311.0    NaN  191.0       NaN  12:46:00  0.0   
           UU           212.0    NaN   11.0    NaN  00:55:00       NaN  0.0   
           UW             NaN  182.0    NaN   82.0       NaN  02:56:00  0.0   
           UX             NaN  203.0    NaN   21.0       NaN  14:18:00  0.0   
           X5           271.0    NaN  311.0    NaN  04:38:00       NaN  0.0   

codiVariable           VVM10 VVM2 VVX10 VVX2  
data       codiEstacio                        
2016-11-01 C9            1.9  NaN   6.3  NaN  
           DL            1.4  NaN   5.0  NaN  
           U7            NaN  1.1   NaN  4.9  
           U9            NaN  0.6   NaN  3.7  
           UA            NaN  0.8   NaN  3.5  
...                      ...  ...   ...  ...  
2021-11-30 US            NaN  0.9   NaN  3.9  
           UU            3.0  NaN   9.3  NaN  
           UW            NaN  1.3   NaN  8.2  
           UX            NaN  1.0   NaN  5.8  
           X5            4.9  NaN  18.7  NaN  

[6420 rows x 11 columns]

In [39]:
# Crea una carpeta dinsar_clean si no existeix per guardar les dades netejades
if not os.path.exists('data_clean'):
    os.makedirs('data_clean')

# Exportem el dataframe a csv per poder-lo utilitzar en altres scripts a la carpeta data_clean

event_data_all_df.to_csv('data_clean/meteo_event_data_all.csv')

# event_data_all_df.to_csv(os.path.join(path_principal, 'event_data_all.csv'))

### Exemples per importar les dades i per fer consultes

In [40]:
# Importem el dataframe de dades de tots els episodis

# Importar normal
event_data_all_df = pd.read_csv('data_clean/meteo_event_data_all.csv')

event_data_all_df

,data,codiEstacio,DVM10,DVM2,DVVX10,DVVX2,HE10M,HE2M,PPT,VVM10,VVM2,VVX10,VVX2
0,2016-11-01,C9,223.0,NaN,181.0,NaN,17:27:00,NaN,0.0,1.9,NaN,6.3,NaN
1,2016-11-01,DL,273.0,NaN,176.0,NaN,14:43:00,NaN,0.0,1.4,NaN,5.0,NaN
2,2016-11-01,U7,NaN,3.0,NaN,9.0,NaN,08:23:00,0.0,NaN,1.1,NaN,4.9
3,2016-11-01,U9,NaN,268.0,NaN,143.0,NaN,14:06:00,0.0,NaN,0.6,NaN,3.7
4,2016-11-01,UA,NaN,288.0,NaN,354.0,NaN,08:57:00,0.0,NaN,0.8,NaN,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,2021-11-30,US,NaN,311.0,NaN,191.0,NaN,12:46:00,0.0,NaN,0.9,NaN,3.9
6416,2021-11-30,UU,212.0,NaN,11.0,NaN,00:55:00,NaN,0.0,3.0,NaN,9.3,NaN
6417,2021-11-30,UW,NaN,182.0,NaN,82.0,NaN,02:56:00,0.0,NaN,1.3,NaN,8.2
6418,2021-11-30,UX,NaN,203.0,NaN,21.0,NaN,14:18:00,0.0,NaN,1.0,NaN,5.8


In [41]:
# Consultes

event_data_all_df[event_data_all_df['codiEstacio'] == 'U9']

,data,codiEstacio,DVM10,DVM2,DVVX10,DVVX2,HE10M,HE2M,PPT,VVM10,VVM2,VVX10,VVX2
3,2016-11-01,U9,NaN,268.0,NaN,143.0,NaN,14:06:00,0.0,NaN,0.6,NaN,3.7
13,2016-11-02,U9,NaN,317.0,NaN,57.0,NaN,11:45:00,0.0,NaN,0.5,NaN,3.1
23,2016-11-03,U9,NaN,278.0,NaN,141.0,NaN,15:09:00,0.0,NaN,0.5,NaN,2.5
33,2016-11-04,U9,NaN,283.0,NaN,251.0,NaN,08:22:00,0.1,NaN,0.5,NaN,4.4
43,2016-11-05,U9,NaN,304.0,NaN,330.0,NaN,16:31:00,0.3,NaN,1.6,NaN,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6373,2021-11-26,U9,NaN,307.0,NaN,283.0,NaN,16:01:00,0.0,NaN,2.6,NaN,11.8
6383,2021-11-27,U9,NaN,291.0,NaN,283.0,NaN,11:05:00,0.0,NaN,2.4,NaN,12.7
6393,2021-11-28,U9,NaN,303.0,NaN,304.0,NaN,07:26:00,0.0,NaN,4.1,NaN,15.2
6403,2021-11-29,U9,NaN,313.0,NaN,336.0,NaN,19:13:00,0.6,NaN,2.6,NaN,10.3


In [42]:
# Importar amb index de data i codiEstacio
event_data_all_df_mi = pd.read_csv(('data_clean/meteo_event_data_all.csv'), index_col=[0,1])
event_data_all_df_mi

DVM10   DVM2  DVVX10  DVVX2     HE10M      HE2M  PPT  \
data       codiEstacio                                                         
2016-11-01 C9           223.0    NaN   181.0    NaN  17:27:00       NaN  0.0   
           DL           273.0    NaN   176.0    NaN  14:43:00       NaN  0.0   
           U7             NaN    3.0     NaN    9.0       NaN  08:23:00  0.0   
           U9             NaN  268.0     NaN  143.0       NaN  14:06:00  0.0   
           UA             NaN  288.0     NaN  354.0       NaN  08:57:00  0.0   
...                       ...    ...     ...    ...       ...       ...  ...   
2021-11-30 US             NaN  311.0     NaN  191.0       NaN  12:46:00  0.0   
           UU           212.0    NaN    11.0    NaN  00:55:00       NaN  0.0   
           UW             NaN  182.0     NaN   82.0       NaN  02:56:00  0.0   
           UX             NaN  203.0     NaN   21.0       NaN  14:18:00  0.0   
           X5           271.0    NaN   311.0    NaN  04:38:00       NaN  0.0   

                        VVM10  VVM2  VVX10  VVX2  
data       codiEstacio                            
2016-11-01 C9             1.9   NaN    6.3   NaN  
           DL             1.4   NaN    5.0   NaN  
           U7             NaN   1.1    NaN   4.9  
           U9             NaN   0.6    NaN   3.7  
           UA             NaN   0.8    NaN   3.5  
...                       ...   ...    ...   ...  
2021-11-30 US             NaN   0.9    NaN   3.9  
           UU             3.0   NaN    9.3   NaN  
           UW             NaN   1.3    NaN   8.2  
           UX             NaN   1.0    NaN   5.8  
           X5             4.9   NaN   18.7   NaN  

[6420 rows x 11 columns]

In [43]:
# Consultes

event_data_all_df_mi.xs('U9', level='codiEstacio')

event_data_all_df_mi.xs('2016-11-26', level='data')



,DVM10,DVM2,DVVX10,DVVX2,HE10M,HE2M,PPT,VVM10,VVM2,VVX10,VVX2
codiEstacio,,,,,,,,,,,
C9,330.0,NaN,263.0,NaN,14:05:00,NaN,41.2,1.6,NaN,8.7,NaN
DL,73.0,NaN,54.0,NaN,12:53:00,NaN,0.0,4.4,NaN,13.7,NaN
U7,NaN,345.0,NaN,110.0,NaN,14:07:00,50.2,NaN,1.0,NaN,5.7
U9,NaN,13.0,NaN,48.0,NaN,13:54:00,4.1,NaN,1.3,NaN,7.9
UA,NaN,51.0,NaN,62.0,NaN,14:24:00,2.7,NaN,2.5,NaN,10.3
US,NaN,40.0,NaN,22.0,NaN,11:53:00,5.2,NaN,1.9,NaN,10.0
UU,329.0,NaN,77.0,NaN,14:12:00,NaN,7.2,3.5,NaN,13.8,NaN
UW,NaN,17.0,NaN,81.0,NaN,13:35:00,3.5,NaN,2.3,NaN,8.3
UX,NaN,32.0,NaN,310.0,NaN,14:11:00,53.7,NaN,0.6,NaN,6.0
